<div class="alert alert-block alert-info">
<b>Deadline:</b> March 20, 2024 (Wednesday) 23:00
</div>

# Exercise 2. Recommender system

In this exercise, your task is to design a recommender system.

## Learning goals:
* Practise tuning a neural network model by using different regularization methods.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import tools
import data

In [2]:
skip_training = False  # Set this flag to True before validation and submission

In [3]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

import tools, warnings
warnings.showwarning = tools.customwarn

In [5]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = "."

In [6]:
# Select the device for training (use GPU if you have one)
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [7]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## Ratings dataset

We will train the recommender system on the dataset in which element consists of three values:
* `user_id` - id of the user (the smallest user id is 1)
* `item_id` - id of the item (the smallest item id is 1)
* `rating` - rating given by the user to the item (ratings are integer numbers between 1 and 5).

The recommender system need to predict the rating for any given pair of `user_id` and `item_id`.

We measure the quality of the predicted ratings using the mean-squared error (MSE) loss:
$$
  \frac{1}{N}\sum_{i=1}^N (r_i - \hat{r}_i)^2
$$
where $r_i$ is a real rating and $\hat{r}_i$ is a predicted one.

Note: The predicted rating $\hat{r}_i$ does not have to be an integer number.

In [8]:
trainset = data.RatingsData(root=data_dir, train=True)
testset = data.RatingsData(root=data_dir, train=False)

In [9]:
# Print one sample from the dataset
x = trainset[0]
print(f'user_id={x[0]}, item_id={x[1]}, rating={x[2]}')

user_id=1, item_id=1, rating=5


# Model

You need to design a recommender system model with the API described in the cell below.

Hints on the model architecture:
* You need to use [torch.nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html?highlight=embedding#torch.nn.Embedding) layer to convert inputs `user_ids` and `item_ids` into reasonable representations. The idea of the embedding layer is that we want to represent similar users with values that are close to each other. The original representation as integers is not good for that. By using the embedding layer, we can learn such useful representations automatically.

### Model tuning

In this exercise, you need to tune the architecture of your model to achieve the best performance on the provided test set. You will notice that overfitting is a severe problem for this data: The model can easily overfit the training set producing poor accuracy on the out-of-training (test) data.

You need to find an optimal combination of the hyperparameters, with some hyperparameters corresponding to the regularization techniques that we studied in the lecture.

The hyperparameters that you are advised to consider:
* Learning rate value and learning rate schedule (decresing the learning rate often has positive effect on the model performance)
* Number of training epochs
* Network size
* Weight decay
* Early stopping
* Dropout
* Increase amount of data:
  * Data augmentation
  * Injecting noise

You can tune the hyperparameters by, for example, grid search, random search or manual tuning. In that case, you can use `architecture` argument to specify the hyperparameters that define the architecture of your network. After you have tuned the hyperparameters, set the default value of this argument to the optimal set of the hyparameters so that the best architecture is used in the accuracy tests.

Note:
* The number of points that you will get from this exercise depends on the MSE loss on the test set:
  * below 1.00: 1 point
  * below 0.95: 2 points
  * below 0.92: 3 points
  * below 0.90: 4 points
  * below 0.89: 5 points
  * below 0.88: 6 points 

In [54]:
class RecommenderSystem(nn.Module):
    def __init__(self, n_users, n_items,
                 architecture=None  # If you want to tune the hyperparameters automatically (e.g. using random
                                    # search), use this argument to specify the hyperparameters that define the
                                    # architecture of your network. After you have tuned the hyperparameters,
                                    # set the default value of this argument to the optimal set of the hyparameters
                                    # so that the best architecture is used in the accuracy tests.
                ):
        """
        Args:
          n_users: Number of users.
          n_items: Number of items.
        """
        # YOUR CODE HERE
        super(RecommenderSystem, self).__init__()
        self.embeddingusers = nn.Embedding(n_users + 1, 10)
        self.embeddingitems = nn.Embedding(n_items + 1, 10)
        self.hidden = nn.Linear(20, 25)
        self.relu = nn.ReLU()
        self.output = nn.Linear(25, 1)
        
        
    def forward(self, user_ids, item_ids):
        """
        Args:
          user_ids of shape (batch_size): User ids (starting from 1).
          item_ids of shape (batch_size): Item ids (starting from 1).
        
        Returns:
          outputs of shape (batch_size): Predictions of ratings.
        """
        # YOUR CODE HERE
        user_embedding = self.embeddingusers(user_ids)
        item_embedding = self.embeddingitems(item_ids)
        x = torch.cat([user_embedding, item_embedding], dim=1)
        x = self.hidden(x)
        x = self.relu(x)
        x = self.output(x)
        return x.squeeze()
        


You can test the shapes of the model outputs using the function below.

In [55]:
def test_RecommenderSystem_shapes():
    n_users, n_items = 100, 1000
    model = RecommenderSystem(n_users, n_items)
    batch_size = 10
    user_ids = torch.arange(1, batch_size+1)
    item_ids = torch.arange(1, batch_size+1)
    output = model(user_ids, item_ids)
    print(output.shape)
    assert output.shape == torch.Size([batch_size]), "Wrong output shape."
    print('Success')

test_RecommenderSystem_shapes()

torch.Size([10])
Success


In [56]:
# This cell is reserved for testing

## Train the model

You need to train a recommender system using **only the training data.** Please use the test set to select the best model: the model that generalizes best to out-of-training data.

**IMPORTANT**:
* During testing, the predictions are produced by `predictions = model(user_ids, item_ids)` with the `user_ids` and `item_ids` loaded from `RatingsData`.
* There is a size limit of 30Mb for saved models.

In [67]:
# Create the model
# IMPORTANT: the default value of the architecture argument should define your best model.
model = RecommenderSystem(trainset.n_users, trainset.n_items)

In [64]:
# This is the function to compute the loss:
def compute_loss(mlp, user_id, item_id, rating):
    mlp.eval()
    with torch.no_grad():
        outputs = mlp.forward(user_id, item_id)
        loss = F.mse_loss(outputs, rating)
        return loss.cpu().numpy()

# This is the function to print the progress during training
def print_progress(epoch, train_error, val_error):
    print('Epoch {}: Train error: {:.4f}, Test error: {:.4f}'.format(
        epoch, train_error, val_error))

In [65]:
train_user_ids = trainset[0][2]
print(train_user_ids)

tensor(5)


In [66]:
# Implement the training loop in this cell
if not skip_training:
    # YOUR CODE HERE
    n_epochs = 500
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)

    train_user_ids, train_item_ids, train_ratings = trainset[:]
    train_user_ids = train_user_ids.to(torch.long)
    train_item_ids = train_item_ids.to(torch.long)
    train_ratings = train_ratings.to(torch.float32)
    test_user_ids, test_item_ids, test_ratings = testset[:]
    test_user_ids = test_user_ids.to(torch.long)
    test_item_ids = test_item_ids.to(torch.long)
    test_ratings = test_ratings.to(torch.float32)
    # train_user_ids = train_user_ids.to(device)
    # train_item_ids = train_item_ids.to(device)
    # train_ratings = train_ratings.to(device)

    for epoch in range(n_epochs):

        optimizer.zero_grad()
        outputs = model.forward(train_user_ids, train_item_ids)
        loss = F.mse_loss(outputs, train_ratings)
        loss.backward()
        optimizer.step()

        if (epoch+1) % 20 == 0:
            print_progress(epoch, compute_loss(model, train_user_ids, train_item_ids, train_ratings), compute_loss(model, test_user_ids, test_item_ids, test_ratings))



    

Epoch 19: Train error: 2.6711, Test error: 2.7243
Epoch 39: Train error: 1.4941, Test error: 1.5266
Epoch 59: Train error: 1.2665, Test error: 1.3186
Epoch 79: Train error: 1.1553, Test error: 1.2123
Epoch 99: Train error: 1.0735, Test error: 1.1244
Epoch 119: Train error: 1.0174, Test error: 1.0657
Epoch 139: Train error: 0.9682, Test error: 1.0148
Epoch 159: Train error: 0.9234, Test error: 0.9691
Epoch 179: Train error: 0.8873, Test error: 0.9330
Epoch 199: Train error: 0.8626, Test error: 0.9085
Epoch 219: Train error: 0.8482, Test error: 0.8946
Epoch 239: Train error: 0.8399, Test error: 0.8862
Epoch 259: Train error: 0.8351, Test error: 0.8811
Epoch 279: Train error: 0.8319, Test error: 0.8779
Epoch 299: Train error: 0.8299, Test error: 0.8757
Epoch 319: Train error: 0.8284, Test error: 0.8742
Epoch 339: Train error: 0.8273, Test error: 0.8731
Epoch 359: Train error: 0.8263, Test error: 0.8723
Epoch 379: Train error: 0.8254, Test error: 0.8718
Epoch 399: Train error: 0.8247, Test

In [68]:
def train(weight_decay=0.01, epochs=350):
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=weight_decay) # weight decay
    criterion = nn.MSELoss()
    
    for i in range(epochs):
        model.train()
        optimizer.zero_grad()
        user_ids, item_ids, ratings = trainset[:]
        user_ids = user_ids.to(torch.long)
        item_ids = item_ids.to(torch.long)
        ratings = ratings.to(torch.float32)
        
        # injecting noise into ratings
        std = 0.1
        ratings += torch.randn_like(ratings) * std
        
        # clamping the ratings between 1 and 5
        
        ratings = torch.clamp(ratings, min=1, max=5)
        
        outputs = model(user_ids, item_ids)
        training_loss = criterion(outputs, ratings)
        
        model.eval()
        with torch.no_grad():
            user_ids, item_ids, ratings = zip(*testset)
            user_ids = torch.tensor(user_ids, dtype=torch.long)
            item_ids = torch.tensor(item_ids, dtype=torch.long)
            ratings = torch.tensor(ratings, dtype=torch.float32)
            validation_outputs = model(user_ids, item_ids)
            validation_loss = criterion(validation_outputs, ratings)
    
        print("Epoch:", i + 1, "training loss:", training_loss, "validation loss:", validation_loss)
        
        training_loss.backward()
        optimizer.step()

In [ ]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
# Set confirm=False if you do not want to be asked for confirmation before saving.
if not skip_training:
    tools.save_model(model, 'recsys.pth', confirm=True)

In [ ]:
# This cell loads your best model
if skip_training:
    model = RecommenderSystem(trainset.n_users, trainset.n_items)
    tools.load_model(model, 'recsys.pth', device)

The next cell tests the accuracy of your best model. It is enough to submit .pth files.

**IMPORTANT**:
* During testing, the predictions are produced by `predictions = model(user_ids, item_ids)` with the `user_ids` and `item_ids` loaded from `RatingsData`.
* There is a size limit of 30Mb for saved models. Please make sure that your model loads in the cell above.

In [ ]:
# This cell tests the accuracy of your best model.

In [ ]:
# This cell is reserved for grading

In [ ]:
# This cell is reserved for grading

In [ ]:
# This cell is reserved for grading

In [ ]:
# This cell is reserved for grading

In [ ]:
# This cell is reserved for grading

In [ ]:
# This cell is reserved for grading